### Basket Combinations

`PICNIC_BASKET1` contains three products: 

1. Six (6) `CROISSANTS`
2. Three (3) `JAMS`
3. One (1) `DJEMBE`

`PICNIC_BASKET2` contains just two products: 

1. Four (4) `CROISSANTS`
2. Two (2) `JAMS`

Aside from the Picnic Baskets, you can now also trade the three products individually on the island exchange. 

Position limits for the newly introduced products:

- `CROISSANT`: 250
- `JAM`: 350
- `DJEMBE`: 60
- `PICNIC_BASKET1`: 60
- `PICNIC_BASKET2`: 100

there are several equivalent trades we can make to get to same composition. for example,

b1 = 6c + 3j + 1d

b2 = 4c + 2j + 0d

b1 = b2 + 2c + 1j + 1d

2b1 = 3b2 + 2d

all of these are equivalent

so we can buy LHS and sell RHS to capture spread

there are thousands of unique combinations like this, how do we determine which ones to examine? do we examine all of them? build a systematic way of examining them?

some approaches can include:

- at runtime, we only examine combinations that're possible to make using current order depth volume
- if we want to examine larger volumes, we'd have to price in basket lower/higher using worse price levels
- we also have our own position to take into account, we can try to continually examine combinations that match our current position and different ways to get to "synthetic netural" by trading synthetic equivalent to current position, thus capturing another spread
- we can also try to swing as much as possible by effectively capturing optimal spread that moves us closest to position limit on other direction, to maximize our trading volume

for analysis:

- different spreads likely have different patterns, with some possibly reverting faster / more stably while others are more volatile
- it's possible that different spreads patterns vary across times and days, making it more profitable to trade certain spreads over others

theory vs practicality:

- cannot examine every combination, have to focus on certain combination
- can try to find every matching combination at every timestamp for analysis and plot interval from min and max spread possible at timestamp

todo:

- synth-spread-generator:
    - to begin, use the best bid and ask volume for each tradeable product to generate every synth combination resulting in equivalent position
        - for each synth, get the execution price to form that synth
        - for each equivalent position, get min and max execution price for synth, use this to calculate spread
        - get min spread and max spread across all equivalent positions (which theoretically should be 0)
    - next steps: take volume and other price levels into consideration


concerns:
- synth-spread-generator doesn't take volume into consideration


todo above is big, should set milestones towards that with minimum viable implementations that're profitable

if we can't do the entire above task, what're the baby steps towards it?

- look at a small subet of combinations which have small position sizes that make it likely globally tradeable
- find combinations that revert frequently and have mean spread closest to 0 across all days, these would be easiest to trade
    - can do this in analysis
- build trader to trade these synth spreads

In [16]:
from tqdm import tqdm

In [17]:
def find_equivalent_basket_combinations(limits):
    # Define the composition of each basket
    basket1 = {"croissants": 6, "jams": 3, "djembes": 1}
    basket2 = {"croissants": 4, "jams": 2, "djembes": 0}
    
    # Initialize a dictionary to track unique combinations by their total products
    unique_combinations = {}
    
    # Function to check if a combination is valid (all products within limits)
    def is_valid_combination(c, j, d, b1, b2):
        return (
            -limits["croissants"] <= c <= limits["croissants"] and
            -limits["jams"] <= j <= limits["jams"] and
            -limits["djembes"] <= d <= limits["djembes"] and
            -limits["basket1"] <= b1 <= limits["basket1"] and
            -limits["basket2"] <= b2 <= limits["basket2"]
        )
    
    # Function to calculate total products in a combination
    def get_product_totals(c, j, d, b1, b2):
        total_c = c + (basket1["croissants"] * b1) + (basket2["croissants"] * b2)
        total_j = j + (basket1["jams"] * b1) + (basket2["jams"] * b2) 
        total_d = d + (basket1["djembes"] * b1) + (basket2["djembes"] * b2)
        return (total_c, total_j, total_d)
    
    # Generate all possible combinations within limits
    for b1 in tqdm(range(-limits["basket1"], limits["basket1"] + 1)):
        for b2 in range(-limits["basket2"], limits["basket2"] + 1):
            for c in range(-limits["croissants"], limits["croissants"] + 1, 2):  # Step by 10 to reduce computation
                for j in range(-limits["jams"], limits["jams"] + 1, 3):  # Step by 10 to reduce computation
                    for d in range(-limits["djembes"], limits["djembes"] + 1):
                        if is_valid_combination(c, j, d, b1, b2):
                            # Calculate the total products
                            totals = get_product_totals(c, j, d, b1, b2)
                            
                            # Add this combination to our dictionary
                            if totals not in unique_combinations:
                                unique_combinations[totals] = []
                            
                            unique_combinations[totals].append((c, j, d, b1, b2))


    return unique_combinations

# Define the limits
limits = {
    "croissants": 250,
    "jams": 350, 
    "djembes": 60,
    "basket1": 60,
    "basket2": 100
}

# Since the full computation would be very large, let's use a reduced set of limits for demonstration
demo_limits = {
    "croissants": 50,  # Reduced from 250
    "jams": 30,        # Reduced from 350
    "djembes": 10,     # Reduced from 60
    "basket1": 10,     # Reduced from 60
    "basket2": 10      # Reduced from 100
}

# Uncomment the line below to run with full limits (warning: this will take a long time and use a lot of memory)
# find_equivalent_basket_combinations(limits)

# Run with reduced limits for demonstration
unique_combinations = find_equivalent_basket_combinations(demo_limits)


100%|██████████| 21/21 [00:11<00:00,  1.79it/s]


In [20]:
combinations = list(unique_combinations.items())

In [28]:
valid_combinations = []
for i in range(len(combinations)):
    pos, comb = combinations[i]
    if len(comb) > 1:
        # print(comb)
        valid_combinations.append(comb)

In [38]:
def combinations_h(l):
    avg_q = 0
    for c in l:
        for p in c:
            avg_q += abs(p)
    avg_q /= len(l)
    return avg_q

min(valid_combinations, key=combinations_h)

[(26, 27, 3, -10, 0),
 (14, 21, 3, -10, 3),
 (2, 15, 3, -10, 6),
 (-10, 9, 3, -10, 9),
 (32, 30, 2, -9, -3),
 (20, 24, 2, -9, 0),
 (8, 18, 2, -9, 3),
 (-4, 12, 2, -9, 6),
 (-16, 6, 2, -9, 9),
 (26, 27, 1, -8, -3),
 (14, 21, 1, -8, 0),
 (2, 15, 1, -8, 3),
 (-10, 9, 1, -8, 6),
 (-22, 3, 1, -8, 9),
 (32, 30, 0, -7, -6),
 (20, 24, 0, -7, -3),
 (8, 18, 0, -7, 0),
 (-4, 12, 0, -7, 3),
 (-16, 6, 0, -7, 6),
 (-28, 0, 0, -7, 9),
 (26, 27, -1, -6, -6),
 (14, 21, -1, -6, -3),
 (2, 15, -1, -6, 0),
 (-10, 9, -1, -6, 3),
 (-22, 3, -1, -6, 6),
 (-34, -3, -1, -6, 9),
 (32, 30, -2, -5, -9),
 (20, 24, -2, -5, -6),
 (8, 18, -2, -5, -3),
 (-4, 12, -2, -5, 0),
 (-16, 6, -2, -5, 3),
 (-28, 0, -2, -5, 6),
 (-40, -6, -2, -5, 9),
 (26, 27, -3, -4, -9),
 (14, 21, -3, -4, -6),
 (2, 15, -3, -4, -3),
 (-10, 9, -3, -4, 0),
 (-22, 3, -3, -4, 3),
 (-34, -3, -3, -4, 6),
 (-46, -9, -3, -4, 9),
 (20, 24, -4, -3, -9),
 (8, 18, -4, -3, -6),
 (-4, 12, -4, -3, -3),
 (-16, 6, -4, -3, 0),
 (-28, 0, -4, -3, 3),
 (-40, -6, -4, 

In [6]:
len(combinations)

121653

In [ ]:

# # If you want to focus on specific examples:
# def check_specific_example():
#     # Example: 2 BASKET1 = 3 BASKET2 + 2 DJEMBE
#     basket1 = {"croissants": 6, "jams": 3, "djembes": 1}
#     basket2 = {"croissants": 4, "jams": 2, "djembes": 0}
    
#     # Calculate totals for each combination
#     combo1_totals = (
#         0 + (2 * basket1["croissants"]) + (0 * basket2["croissants"]),
#         0 + (2 * basket1["jams"]) + (0 * basket2["jams"]),
#         0 + (2 * basket1["djembes"]) + (0 * basket2["djembes"])
#     )
    
#     combo2_totals = (
#         0 + (0 * basket1["croissants"]) + (3 * basket2["croissants"]),
#         0 + (0 * basket1["jams"]) + (3 * basket2["jams"]),
#         2 + (0 * basket1["djembes"]) + (0 * basket2["djembes"])
#     )
    
#     print("\nChecking specific example: 2 BASKET1 = 3 BASKET2 + 2 DJEMBE")
#     print(f"Combination 1 (2 BASKET1) totals: {combo1_totals[0]} croissants, {combo1_totals[1]} jams, {combo1_totals[2]} djembes")
#     print(f"Combination 2 (3 BASKET2 + 2 DJEMBE) totals: {combo2_totals[0]} croissants, {combo2_totals[1]} jams, {combo2_totals[2]} djembes")
#     print(f"Are they equivalent? {combo1_totals == combo2_totals}")

# check_specific_example()